## Лекция 4. Глобальные переменные. Так ли страшен чёрт?

### Почему глобальные переменные считаются плохой практикой?

Глобальные переменные часто считаются плохой практикой программирования по нескольким причинам:

1. **Непредсказуемость поведения:** Глобальные переменные делают поведение функций непредсказуемым. Функция может работать по-разному в зависимости от состояния глобальных переменных, что усложняет понимание кода.

2. **Скрытые зависимости:** Когда функция использует глобальные переменные, это не очевидно из её сигнатуры. Другие разработчики не могут понять, какие данные нужны функции, просто посмотрев на её параметры.

3. **Сложность отладки:** При отладке сложно понять, где и когда изменилась глобальная переменная. Это может привести к неожиданным ошибкам, которые трудно воспроизвести.

4. **Проблемы в многопоточности:** В многопоточных приложениях глобальные переменные могут вызывать гонки условий, когда несколько потоков одновременно изменяют одну переменную.

5. **Нарушение принципов чистых функций:** Функции, использующие глобальные переменные, не являются чистыми - их результат зависит не только от входных параметров, но и от внешнего состояния.

Однако, глобальные переменные могут быть полезны в некоторых случаях:

* **Константы:** постоянные величины, которые не изменяются в ходе выполнения программы.
* **Конфигурационные параметры:** параметры, которые задают поведение программы.

и др.

Чтобы избежать проблем с глобальными переменными, можно использовать следующие подходы:

* **Классы:** Классы позволяют инкапсулировать данные и методы, работающие с этими данными. Вместо глобальных переменных используются атрибуты объекта.
* **Модули:** Модули в Python создают своё пространство имён. Переменные в модуле доступны через импорт, что делает зависимости явными.
* **Замыкания:** Замыкания позволяют сохранять состояние между вызовами функции без использования глобальных переменных.
* **Конфигурационные файлы:** Создание специальных объектов для хранения конфигурации делает код более структурированным и тестируемым.
* **Dependency Injection:** Передача зависимостей через параметры функций делает код более предсказуемым и тестируемым.

### Пример 1. Счётчик (counter)

**Плохой подход:**

In [ ]:
counter = 0

def increment():
    global counter
    counter += 1

def reset():
    global counter
    counter = 0

def get_counter():
    return counter

increment()
increment()
print(get_counter())
reset()
print(get_counter())


**Подход через создание класса**

In [ ]:
class Counter:
    def __init__(self) -> None:
        self.value = 0
    
    def increment(self):
        self.value += 1

    def reset(self):
        self.value = 0
    
    def get_value(self):
        return self.value
    
counter_1 = Counter()
counter_2 = Counter()

for _ in range(5):
    counter_1.increment()

for _ in range(3):
    counter_2.increment()

print(counter_1.get_value())
print(counter_2.get_value())

counter_1.reset()
counter_2.reset()

print(counter_1.get_value())
print(counter_2.get_value())

**Подход через замыкание**

Вспомните LEGB. 

In [ ]:
def create_counter():
    count = 0
    
    def increment():
        nonlocal count
        count += 1
        return count
    
    def reset():
        nonlocal count
        count = 0
    
    def get_value():
        return count
    
    return increment, reset, get_value

increment, reset, get_value = create_counter()
increment_2, reset_2, get_value_2 = create_counter()

increment()
increment()
print(get_value())

increment_2()

print(get_value())
print(get_value_2())


### Пример 2. Конфигурационные параметры.

Часто в коде есть множество констант, которые используются для конфигурирования поведения кода. Эти константы действительно удобно задавать в виде глобальных переменных. Но есть несколько вопросов:

1. В каком виде эти переменные задать, чтобы при эелании их можно было легко и прозрачно изменить?
2. Как быть уверенным, что в ходе исполнения кода эти константы не изменятся?

**Пример задачи**

Мы хотим замоделировать случайное блуждание точки на плоскости. Для этого мы зададим размеры нашего поля (например, 10 х 10 точек). Также, мы зададим изначальные координаты точки (например, [0, 0]) и количество шагов в нашем моделировании (например, 100).

Для симуляции случайных блужданий мы используем библиотеку `random`. И зададим насколько шагов максимум за один шаг по осям *x* и *y* может двигаться наша точка. 

По итогу давайте выведем, сколько раз за симуляцию наша точка снова оказалась в исходном положении.


**Вариант 1. Всё через глобальные переменные.**

In [ ]:
from random import randint

class Counter:
    def __init__(self) -> None:
        self.value = 0
    
    def increment(self):
        self.value += 1

    def reset(self):
        self.value = 0
    
    def get_value(self):
        return self.value

# Параметры поля
LEFT_X = -5
RIGHT_X = 5
TOP_Y = 5
BOTTOM_Y = -5

# Изначальное положение точки
X0 = 0
Y0 = 0

# Количество шагов симуляции
NUM_STEPS = 1000

# Максимальная длина шага
MAX_STEP_X = 2
MAX_STEP_Y = 2


# Основное тело программы
x = X0
y = Y0
counter = Counter()

for _ in range(NUM_STEPS):
    x_step = randint(- MAX_STEP_X, MAX_STEP_X)
    while (x + x_step < LEFT_X) or (x + x_step > RIGHT_X):
        x_step = randint(- MAX_STEP_X, MAX_STEP_X)

    y_step = randint(- MAX_STEP_Y, MAX_STEP_Y)
    while (y + y_step < BOTTOM_Y) or (y + y_step > TOP_Y):
        y_step = randint(- MAX_STEP_Y, MAX_STEP_Y)

    x += x_step
    y += y_step

    if x == X0 and y == Y0:
        counter.increment()

print(counter.get_value())


Это уже неплохо работающая программа. Чтобы выделить константы, мы написали их в начале кода и обозначили заглавными буквами (это общепринятая в Python практика). Но, если наш код большой и сложный, любая из этих констант случайно может быть изменеа, а мы этого даже не заметим. 

С другой стороны, представьте, что нам нужно провести несколько таких экспериментов. И для каждого из них нам придётся менять конфигурационные параметы. Это каждый раз надо лезь в исходный код. Но мы можем забыть что-то поменять или случайно изменить код, который мы не планировали.

Попробуем улучшить нашу программу.

**Код с использованием неизменяемых типов данных**

In [ ]:
from random import randint

class Counter:
    def __init__(self) -> None:
        self.value = 0
    
    def increment(self):
        self.value += 1

    def reset(self):
        self.value = 0
    
    def get_value(self):
        return self.value

# Параметры поля
FIELD_X = (-5, 5)
FIELD_Y = (-5, 5)

# Изначальное положение точки
XY_0 = (0, 0)

# Количество шагов симуляции
NUM_STEPS = (1000,)

# Максимальная длина шага
MAX_STEPS = (2, 2)


# Основное тело программы
x = XY_0[0]
y = XY_0[1]
counter = Counter()

for _ in range(NUM_STEPS[0]):
    x_step = randint(- MAX_STEPS[0], MAX_STEPS[0])
    while (x + x_step < FIELD_X[0]) or (x + x_step > FIELD_X[1]):
        x_step = randint(- MAX_STEPS[0], MAX_STEPS[0])

    y_step = randint(- MAX_STEPS[1], MAX_STEPS[1])
    while (y + y_step < FIELD_Y[0]) or (y + y_step > FIELD_Y[1]):
        y_step = randint(- MAX_STEPS[1], MAX_STEPS[1])

    x += x_step
    y += y_step

    if x == XY_0[0] and y == XY_0[1]:
        counter.increment()

print(counter.get_value())


# Проверим неизменяемость наших констант:
try:
    FIELD_X[0] = 8
except Exception as e:
    print(e)


Также можно использовать специальные классы для хранени данных.

In [ ]:
from dataclasses import dataclass
from random import randint

class Counter:
    def __init__(self) -> None:
        self.value = 0
    
    def increment(self):
        self.value += 1

    def reset(self):
        self.value = 0
    
    def get_value(self):
        return self.value


@dataclass(frozen=True)
class Config:
    # Параметры поля
    LEFT_X   : int = -5
    RIGHT_X  : int = 5
    TOP_Y    : int = 5
    BOTTOM_Y : int = -5

    # Изначальное положение точки
    X0 : int = 0
    Y0 : int = 0

    # Количество шагов симуляции
    NUM_STEPS : int = 1000

    # Максимальная длина шага
    MAX_STEP_X : int = 2
    MAX_STEP_Y : int = 2

config = Config()

# Основное тело программы
x = config.X0
y = config.Y0
counter = Counter()

for _ in range(config.NUM_STEPS):
    x_step = randint(- config.MAX_STEP_X, config.MAX_STEP_X)
    while (x + x_step < config.LEFT_X) or (x + x_step > config.RIGHT_X):
        x_step = randint(- config.MAX_STEP_X, config.MAX_STEP_X)

    y_step = randint(- config.MAX_STEP_Y, config.MAX_STEP_Y)
    while (y + y_step < config.BOTTOM_Y) or (y + y_step > config.TOP_Y):
        y_step = randint(- config.MAX_STEP_Y, config.MAX_STEP_Y)

    x += x_step
    y += y_step

    if x == config.X0 and y == config.Y0:
        counter.increment()

print(counter.get_value())

# Проверяем на неизменяемость

try:
    config.NUM_STEPS += 1
except Exception as e:
    print(e)


Более "мягкий" способ решения -- поместить все константы внутрь функции. Тогда они станут локальными константами, и мы будем уверены, что они не будут использованы вне кода функции. Это хорошая стратегия, если константы нужны нам для работы только одной функции.

In [ ]:
from random import randint

class Counter:
    def __init__(self) -> None:
        self.value = 0
    
    def increment(self):
        self.value += 1

    def reset(self):
        self.value = 0
    
    def get_value(self):
        return self.value
    

def simulate_random_walk():
    # Параметры поля
    LEFT_X = -5
    RIGHT_X = 5
    TOP_Y = 5
    BOTTOM_Y = -5

    # Изначальное положение точки
    X0 = 0
    Y0 = 0

    # Количество шагов симуляции
    NUM_STEPS = 1000

    # Максимальная длина шага
    MAX_STEP_X = 2
    MAX_STEP_Y = 2


    # Основное тело программы
    x = X0
    y = Y0
    counter = Counter()

    for _ in range(NUM_STEPS):
        x_step = randint(- MAX_STEP_X, MAX_STEP_X)
        while (x + x_step < LEFT_X) or (x + x_step > RIGHT_X):
            x_step = randint(- MAX_STEP_X, MAX_STEP_X)

        y_step = randint(- MAX_STEP_Y, MAX_STEP_Y)
        while (y + y_step < BOTTOM_Y) or (y + y_step > TOP_Y):
            y_step = randint(- MAX_STEP_Y, MAX_STEP_Y)

        x += x_step
        y += y_step

        if x == X0 and y == Y0:
            counter.increment()

    print(counter.get_value())


simulate_random_walk()

# Проверяем на неизменяемость

try:
    NUM_STEPS += 1
except Exception as e:
    print(e)

Но если мы засунем все константы внутрь функции, то мы не сможем их удобно менять. Нам придётся лезть внутрь кода функции. 

Можно попробовать обозначить все константы как параметры функции. Это гарантирует нам, что они будут применены только внутри функции. И также мы сможем их менять не меняя кода внутри функции.

In [ ]:
from random import randint

class Counter:
    def __init__(self) -> None:
        self.value = 0
    
    def increment(self):
        self.value += 1

    def reset(self):
        self.value = 0
    
    def get_value(self):
        return self.value
    

def simulate_random_walk(
    LEFT_X = -5,
    RIGHT_X = 5,
    TOP_Y = 5,
    BOTTOM_Y = -5,
    X0 = 0,
    Y0 = 0,
    NUM_STEPS = 1000,
    MAX_STEP_X = 2,
    MAX_STEP_Y = 2,
):

    # Основное тело программы
    x = X0
    y = Y0
    counter = Counter()


    for _ in range(NUM_STEPS):
        x_step = randint(- MAX_STEP_X, MAX_STEP_X)
        while (x + x_step < LEFT_X) or (x + x_step > RIGHT_X):
            x_step = randint(- MAX_STEP_X, MAX_STEP_X)

        y_step = randint(- MAX_STEP_Y, MAX_STEP_Y)
        while (y + y_step < BOTTOM_Y) or (y + y_step > TOP_Y):
            y_step = randint(- MAX_STEP_Y, MAX_STEP_Y)

        x += x_step
        y += y_step

        if x == X0 and y == Y0:
            counter.increment()

    print(counter.get_value())


simulate_random_walk()

# Проверяем на неизменяемость

try:
    NUM_STEPS += 1
except Exception as e:
    print(e)

Но согласитесь, если у функции так много параметров, то код становится тяжелочитаемым и в нём очень легко запутаться. Особенно при передаче новых параметров в функцию.

Чтобы жить было проще и удобнее, существуют конфигурационные файлы, про которые мы и поговорим дальше.

**конфигурационный python файл**

In [ ]:
from random import randint

class Counter:
    def __init__(self) -> None:
        self.value = 0
    
    def increment(self):
        self.value += 1

    def reset(self):
        self.value = 0
    
    def get_value(self):
        return self.value
    

def simulate_random_walk():
    from config_simulation import LEFT_X, RIGHT_X, TOP_Y, BOTTOM_Y, NUM_STEPS
    from config_simulation import MAX_STEP_X, MAX_STEP_Y, X0, Y0


    # Основное тело программы
    x = X0
    y = Y0
    counter = Counter()

    for _ in range(NUM_STEPS):
        x_step = randint(- MAX_STEP_X, MAX_STEP_X)
        while (x + x_step < LEFT_X) or (x + x_step > RIGHT_X):
            x_step = randint(- MAX_STEP_X, MAX_STEP_X)

        y_step = randint(- MAX_STEP_Y, MAX_STEP_Y)
        while (y + y_step < BOTTOM_Y) or (y + y_step > TOP_Y):
            y_step = randint(- MAX_STEP_Y, MAX_STEP_Y)

        x += x_step
        y += y_step

        if x == X0 and y == Y0:
            counter.increment()

    print(counter.get_value())


simulate_random_walk()

# Проверяем на неизменяемость

try:
    NUM_STEPS += 1
except Exception as e:
    print(e)

**.json конфиг**

In [ ]:
from random import randint

class Counter:
    def __init__(self) -> None:
        self.value = 0
    
    def increment(self):
        self.value += 1

    def reset(self):
        self.value = 0
    
    def get_value(self):
        return self.value
    


import json
def simulate_random_walk(config_path:str):
    with open(config_path, "r") as f:
        config = json.load(f)

    # Основное тело программы
    x = config["X0"]
    y = config["Y0"]
    counter = Counter()

    for _ in range(config["NUM_STEPS"]):
        x_step = randint(- config["MAX_STEP_X"], config["MAX_STEP_X"])
        while (x + x_step < config["LEFT_X"]) or (x + x_step > config["RIGHT_X"]):
            x_step = randint(- config["MAX_STEP_X"], config["MAX_STEP_X"])

        y_step = randint(- config["MAX_STEP_Y"], config["MAX_STEP_Y"])
        while (y + y_step < config["BOTTOM_Y"]) or (y + y_step > config["TOP_Y"]):
            y_step = randint(- config["MAX_STEP_Y"], config["MAX_STEP_Y"])

        x += x_step
        y += y_step

        if x == config["X0"] and y == config["Y0"]:
            counter.increment()

    print(counter.get_value())

config_path = "config.json"
simulate_random_walk(config_path)

# Проверяем на неизменяемость
try:
    NUM_STEPS += 1
except Exception as e:
    print(e)

**.yaml конфиг**

In [ ]:
!pip install pyyaml

In [ ]:
from random import randint

class Counter:
    def __init__(self) -> None:
        self.value = 0
    
    def increment(self):
        self.value += 1

    def reset(self):
        self.value = 0
    
    def get_value(self):
        return self.value
    


import yaml
def simulate_random_walk(config_path:str):
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)

    # Основное тело программы
    x = config["X0"]
    y = config["Y0"]
    counter = Counter()

    for _ in range(config["NUM_STEPS"]):
        x_step = randint(- config["MAX_STEP_X"], config["MAX_STEP_X"])
        while (x + x_step < config["LEFT_X"]) or (x + x_step > config["RIGHT_X"]):
            x_step = randint(- config["MAX_STEP_X"], config["MAX_STEP_X"])

        y_step = randint(- config["MAX_STEP_Y"], config["MAX_STEP_Y"])
        while (y + y_step < config["BOTTOM_Y"]) or (y + y_step > config["TOP_Y"]):
            y_step = randint(- config["MAX_STEP_Y"], config["MAX_STEP_Y"])

        x += x_step
        y += y_step

        if x == config["X0"] and y == config["Y0"]:
            counter.increment()

    print(counter.get_value())

config_path = "config.yaml"
simulate_random_walk(config_path)

# Проверяем на неизменяемость
try:
    NUM_STEPS += 1
except Exception as e:
    print(e)

### Семинар 4

**Задача:** 

---

Первый автомобиль стартует с изначальной скоростью `V0_1` и ускорением `a1`. 

Одновременно с ним стартует второй автомобиль с изначальной скоростью `V0_2` и ускорением `a2`. 

Известно, что изначальное расстояние между автомобилями равно `L0`, причём второй автомобиль находится ближе к финишной черте.

Также известно расстояние от первого автомобиля до финиша: `DISTANCE`.

Какой автомобиль первый доберётся до финиша?

---

Напишите код для решения этой задачи, оформив все константы как dataclass. 